In [186]:
import os
from dotenv import load_dotenv

load_dotenv()

# Database connection parameters
DATABASE = os.getenv('DB_NAME')
USER = os.getenv('DB_USER')
PASSWORD = os.getenv('DB_PASSWORD')
HOST = os.getenv('DB_HOST')
PORT = os.getenv('DB_PORT')

TOGETHER_KEY = os.getenv('TOGETHER_API_KEY')
OPENAI_KEY = os.getenv('OPENAI_API_KEY')

In [187]:
from langchain_community.utilities import SQLDatabase

postgres_uri = f"postgresql://{USER}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}"

db = SQLDatabase.from_uri(postgres_uri)
print(db.dialect)
print(db.get_usable_table_names())

result = db.run("SELECT * FROM grades;")
print(result)

postgresql
['cognitive_dimension', 'grades', 'knowledge_dimension', 'literacy_dimension', 'subjects', 'subtopics', 'topics', 'units']
[(6, 'Six'), (7, 'Seven'), (8, 'Eight'), (9, 'Nine'), (10, 'Ten'), (11, 'Eleven'), (12, 'Twelve')]


In [39]:
from typing import Any

from langchain_core.messages import ToolMessage
from langchain_core.runnables import RunnableLambda, RunnableWithFallbacks
from langgraph.prebuilt import ToolNode


def create_tool_node_with_fallback(tools: list) -> RunnableWithFallbacks[Any, dict]:
    """
    Create a ToolNode with a fallback to handle errors and surface them to the agent.
    """
    return ToolNode(tools).with_fallbacks(
        [RunnableLambda(handle_tool_error)], exception_key="error"
    )


def handle_tool_error(state) -> dict:
    error = state.get("error")
    tool_calls = state["messages"][-1].tool_calls
    return {
        "messages": [
            ToolMessage(
                content=f"Error: {repr(error)}\n please fix your mistakes.",
                tool_call_id=tc["id"],
            )
            for tc in tool_calls
        ]
    }

In [41]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit

from langchain_together import ChatTogether

toolkit = SQLDatabaseToolkit(db=db, llm=ChatTogether(
    model="meta-llama/Llama-3.3-70B-Instruct-Turbo",
))
tools = toolkit.get_tools()

# these tools are present in the SQLDatabaseToolkit
list_tables_tool = next(tool for tool in tools if tool.name == "sql_db_list_tables") # list the tables
get_schema_tool = next(tool for tool in tools if tool.name == "sql_db_schema") # show the schema of the specific table

In [290]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI

from typing import List

topic_identifier_system = """Analyze user input and identify the physics topic mentioned in the input. Do not return the input text.
For example:
- "Create questions about velocity" -> "velocity"
- "Explain displacement" -> "displacement"
 
Return only the identified physics topic name as given in the input."""

topic_check = ChatPromptTemplate.from_messages([
   ("system", topic_identifier_system),
   ("placeholder", "{messages}")
]) | ChatTogether(
    model="meta-llama/Llama-3.3-70B-Instruct-Turbo", temperature=0
    )

input = "Create 5 questions on third law of motion"

topic = topic_check.invoke({"messages": [("user", input)]}).content

print(topic)


third law of motion


In [260]:
topic

"newton's forst law"

In [281]:
from langchain_together import ChatTogether
from langchain.prompts import ChatPromptTemplate

def find_matching_topic(topic_of_interest: str) -> str:
    """
    Find most matching topic using LLM reasoning.
    """
    # Get all topics from database
    query = "SELECT topic_name, topic_id FROM topics;"
    result = db.run_no_throw(query)
    
    if not result:
        return "No topics found in database"
    
    print(result)
    
    topics = result
    
    # Initialize LLM
    llm = ChatTogether(
    model="meta-llama/Llama-3.3-70B-Instruct-Turbo", temperature=0
    )
    
    # Create prompt
    prompt = ChatPromptTemplate.from_template(
        """You are a topic matching expert. Find the most similar topic from the available list.

        Topic to match: {topic_of_interest}

        Available topics:
        {topics}

        Return ONLY the exact matching topic_ID and nothing more.
        
        If none match well, return 'NO_MATCH'.
        """
    )
    
    # Create chain and run
    chain = prompt | llm
    
    response = chain.invoke({
        "topic_of_interest": topic_of_interest,
        "topics": "\n".join(topics)
    })
    
    return response.content

# Example usage
topic = "newton's forst law"
matched_topic = find_matching_topic(topic)
print(matched_topic)

[('Relative Motion, Distance, and Displacement', 'OS_PH_HS_02_01'), ('Speed and Velocity', 'OS_PH_HS_02_02'), ('Position vs. Time Graphs', 'OS_PH_HS_02_03'), ('Velocity vs. Time Graphs', 'OS_PH_HS_02_04'), ('Acceleration', 'OS_PH_HS_03_01'), ('Representing Acceleration with Equations and Graphs', 'OS_PH_HS_03_02'), ('Force', 'OS_PH_HS_04_01'), ("Newton's first law of Motion: Inertia", 'OS_PH_HS_04_02'), ("Newton's second law of motion", 'OS_PH_HS_04_03'), ("Newton's Third law of motion", 'OS_PH_HS_04_04'), ("Newton's law of universal gravitation ", 'OS_PH_HS_07_02')]
OS_PH_HS_04_02


In [272]:
# import json
# def get_topic_id(json_string: str) -> str:
#    try:
#        return json.loads(json_string).get('topic_id', '')
#    except json.JSONDecodeError:
#        return ''
   
# topic_id = get_topic_id(matched_topic)

In [256]:
# topic_id

''

In [274]:
topic_id = matched_topic

In [280]:
db.run("""WITH topic_search AS 
(SELECT topic_id FROM topics 
WHERE topic_name ILIKE '%{topic}%') 
SELECT s.subtopic_name, 
s.description, 
s.mathematical_formulation,
s.prerequisites,
s.misconceptions,
s.engineering_applications,
s.cross_cutting_topics,
s.analogies
FROM subtopics s 
JOIN topic_search ts 
ON s.topic_id = ts.topic_id 
LIMIT 10;""")

''

In [211]:
from langchain_core.tools import tool

# this tool can run a SQL query and get the output

@tool
def db_query_tool(query: str) -> str:
    """
    Execute a SQL query against the database and get back the result.
    If the query is not correct, an error message will be returned.
    If an error is returned, rewrite the query, check the query, and try again.
    """
    result = db.run_no_throw(query)
    if not result:
        return "Error: Query failed. Please rewrite your query and try again."
    return result


In [307]:
from langchain_core.prompts import ChatPromptTemplate

query_check_system = """You are a SQL expert with a strong attention to detail.
Double check the PostgreSQL query for common mistakes, including:

- Using NOT IN with NULL values
- Using UNION when UNION ALL should have been used
- Using BETWEEN for exclusive ranges
- Data type mismatch in predicates
- Properly quoting identifiers
- Using the correct number of arguments for functions
- Casting to the correct data type
- Using the proper columns for joins

If there are any of the above mistakes, rewrite the query. If there are no mistakes, just reproduce the original query.

You will call the appropriate tool to execute the query after running this check.

Modify the topic_name in the query based on a similar item in topics"""

# system prompt is query_check_system
query_check_prompt = ChatPromptTemplate.from_messages(
    [("system", query_check_system), ("placeholder", "{messages}")]
)
query_check = query_check_prompt | ChatTogether(
    model="meta-llama/Llama-3.3-70B-Instruct-Turbo", temperature=0
    ).bind_tools(
    [db_query_tool], 
    tool_choice="required"
)

query = f"""WITH topic_search AS 
(SELECT topic_id FROM topics 
WHERE topic_id ILIKE '{matched_topic}') 
SELECT s.subtopic_name, 
s.description, 
s.mathematical_formulation,
s.prerequisites,
s.misconceptions,
s.engineering_applications,
s.cross_cutting_topics,
s.analogies
FROM subtopics s 
JOIN topic_search ts 
ON s.topic_id = ts.topic_id 
LIMIT 10;"""

# user message is the SQL query
message = query_check | (lambda x: db_query_tool(x.tool_calls[0]['args']['query'])) 

context = message.invoke({"messages": [("user", query)]})

db.run(query)


# query = message.tool_calls[0]['args']['query']

'[(\'Mass\', "Mass is a scalar quantity that measures the amount of matter in an object. It is often regarded as a measure of an object\'s inertia, and it determines how much force is required to change the object\'s state of motion. Mass is constant for an object and is independent of its location in the...", None, {\'internal\': [\'Basic Understanding of Matter\'], \'external\': {\'mathematics\': [\'Basic Algebra\', \'Unit Conversion (kilograms, grams)\']}}, [\'Mass and weight are the same.\', \'Mass changes with location.\', \'Mass indicates size or volume\', \'Mass equals inertia\', \'Mass disappears during freefall.\', \'Mass is a vector quantity.\'], [\'In aerospace engineering for determining mass distribution for stability of the aircraft.\', \'Optimizing vehicle mass for fuel efficiency, performance, and safety in crash tests.\', \'Mass distribution of buildings for their stability in civil engineering is determined.\'], ["Newtonian Mechanics:Mass plays a central role in Newto

In [303]:
message

ChatPromptTemplate(input_variables=[], optional_variables=['messages'], input_types={'messages': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annot

In [306]:
context

'Error: Query failed. Please rewrite your query and try again.'

In [169]:
import ast
import json

def format_individual_topics(input_text):
    data = ast.literal_eval(input_text)
    
    # Process each tuple into individual JSONs
    individual_jsons = {}
    
    for topic in data:
        topic_dict = {
            "subtopic_name": topic[0],
            "description": topic[1],
            "mathematical_formulation": topic[2],
            "pre-requisites": topic[3],
            "misconceptions": topic[4],
            "engineering_applications": topic[5],
            "cross_cutting_topics": topic[6],
            "analogies": topic[7]
        }
        
        # Store JSON string for each topic
        individual_jsons[topic[0]] = json.dumps(topic_dict, indent=4)
    
    return individual_jsons


jsons = format_individual_topics(context)
for topic_name, json_str in jsons.items():
    print(f"JSON for {topic_name}:")
    print(json_str)
    print()

JSON for Mass:
{
    "subtopic_name": "Mass",
    "description": "Mass is a scalar quantity that measures the amount of matter in an object. It is often regarded as a measure of an object's inertia, and it determines how much force is required to change the object's state of motion. Mass is constant for an object and is independent of its location in the...",
    "mathematical_formulation": null,
    "pre-requisites": {
        "internal": [
            "Basic Understanding of Matter"
        ],
        "external": {
            "mathematics": [
                "Basic Algebra",
                "Unit Conversion (kilograms, grams)"
            ]
        }
    },
    "misconceptions": [
        "Mass and weight are the same.",
        "Mass changes with location.",
        "Mass indicates size or volume",
        "Mass equals inertia",
        "Mass disappears during freefall.",
        "Mass is a vector quantity."
    ],
    "engineering_applications": [
        "In aerospace engineering

In [ ]:
# Base prompt template for each skill level
prompt_template = """Create one multiple choice question for {skill} level of Bloom's taxonomy for a 9th grade Physics student in India on {topic}. 

Use {context} for accuracy in creating the questions and distractors. 

Specifically use information from the prerequisites, misconceptions, engineering_applications, cross_cutting_topics, analogies for the generation.

Requirements:

- Student should only be able to answer if they've mastered the concept"
- Each distractor must address either: 
    - A specific misconception about {topic} 
    - A prerequisite knowledge gap"
Language and complexity suitable for 9th grade
Physics context and application

Format as the output as a JSON:

Create one multiple choice question for {skill} level of Bloom's taxonomy for a 9th grade Physics student in India on {topic}."
    "Requirements:"
    "Student should only be able to answer if they've mastered the concept"
    "Each distractor must address either: A specific misconception about {topic} or A prerequisite knowledge gap"
    "Language and complexity suitable for 9th grade"
    "Physics context and application"
    "Use {context} for accuracy in creating the questions and distractors."
    "Format as the output as a JSON:

{{{{
   "question": "",
   "skill": ""
   "options": {{"a": "", "b": "", "c": "", "d": ""}},
   "correct": "",
   "explanation": {{
       "correct": "",
       "a": "misconception/prerequisite tested",
       "b": "", "c": "", "d": ""
   }}
}}}}

For {skill} level, ensure:
{skill_requirements}

Make sure there are no additional information being other than the output in the format that is asked for.

"""

# llm = ChatOpenAI(model="gpt-4o",
#                  api_key=os.getenv("OPENAI_API_KEY"))
 
# llm = ChatTogether(
#     model="Qwen/Qwen2.5-72B-Instruct-Turbo"
#     )
 
llm = ChatTogether(
    model="meta-llama/Llama-3.3-70B-Instruct-Turbo"
    )
 
# llm = ChatTogether(
#     model="meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo"
#     )
 
# llm = ChatOpenAI(model="gpt-4o-mini",
#                  api_key=os.getenv("OPENAI_API_KEY"))

# Skill-specific requirements
skill_requirements = {
   "Remember": "Question tests ability to retrieve relevant knowledge from long-term memory.",
   "Understand": "Question tests ability to onstruct meaning from instructional messages, including oral, written, and graphic communication.",
   "Apply": "Question tests ability to carry out or use a procedure in a given situation.",
   "Analyze": "Question tests ability to break material into foundational parts and determine how parts relate to one another and the overall structure or purpose.",
   "Evaluate": "Question tests ability to make judgments based on criteria and standards."
}

skills = ["Remember", "Understand", "Apply", "Analyze", "Evaluate"]

def generate_assessment(topic, context, llm):
   responses = {
       "topic": topic,
       "questions": []
   }
   
   for skill in skills:
       prompt = prompt_template.format(
           skill=skill,
           topic=topic,
           context=context,
           skill_requirements=skill_requirements[skill]
       )
       response = llm.invoke(prompt)
       cleaned_content = response.content.strip()
       if cleaned_content.startswith("```json"):
           cleaned_content = cleaned_content[7:-3]
           
       try:
           question_json = json.loads(cleaned_content)
           responses["questions"].append(question_json)
       except json.JSONDecodeError as e:
           print(response)
           print(f"Error parsing {skill} response")
   
   model_name = llm.model_name.split('/')[-1]
   filename = f"Concept_{topic}_{model_name}.json"
   
   with open(filename, "w") as f:
       json.dump(responses, f, indent=2, ensure_ascii=False)
       
   return responses

final_assessment = generate_assessment(topic, context, llm)
